## An Electrode Design Optimisation Example

A design optimisation example loosely based on work by L.D. Couto available at [[1]](https://doi.org/10.1016/j.energy.2022.125966).

The target is to maximise the gravimetric energy density over a range of possible design parameter values, including for example:

cross-sectional area = height x width (only need change one), electrode widths, particle radii, volume fractions and separator width.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP and upgrade dependencies:

In [ ]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Next, we import the added packages plus any additional dependencies,

In [ ]:
import numpy as np
import pybamm
from pybamm import Parameter

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

Let's fix the random seed in order to generate consistent output during development, although this does not need to be done in practice.

In [ ]:
np.random.seed(8)

## Optimising the Parameters

First, we define the model to be used for the parameter optimisation,

In [ ]:
parameter_set = pybamm.ParameterValues("Chen2020")
parameter_set.update(
    {
        "Electrolyte density [kg.m-3]": Parameter("Separator density [kg.m-3]"),
        "Negative electrode active material density [kg.m-3]": Parameter(
            "Negative electrode density [kg.m-3]"
        ),
        "Negative electrode carbon-binder density [kg.m-3]": Parameter(
            "Negative electrode density [kg.m-3]"
        ),
        "Positive electrode active material density [kg.m-3]": Parameter(
            "Positive electrode density [kg.m-3]"
        ),
        "Positive electrode carbon-binder density [kg.m-3]": Parameter(
            "Positive electrode density [kg.m-3]"
        ),
    },
    check_already_exists=False,
)
model = pybamm.lithium_ion.SPMe()

Next, we define the model parameters for optimisation. Furthermore, PyBOP provides functionality to define a prior for the parameters. The initial parameters values used in the optimisation will be randomly drawn from the prior distribution.

In [ ]:
parameters = [
    pybop.Parameter(
        "Positive electrode thickness [m]",
        prior=pybop.Gaussian(7.56e-05, 0.05e-05),
        bounds=[65e-06, 10e-05],
    ),
    pybop.Parameter(
        "Positive particle radius [m]",
        prior=pybop.Gaussian(5.22e-06, 0.05e-06),
        bounds=[2e-06, 9e-06],
    ),
]

Next, we construct the experiment for design optimisation and the initial state-of-charge,

In [ ]:
experiment = pybamm.Experiment(
    ["Discharge at 1C until 2.5 V (5 seconds period)"],
)

We can now define the output signal, the problem (which combines the model with the dataset) and construct a cost function which in this example is the `GravimetricEnergyDensity()` used to maximise the gravimetric energy density of the cell.

In [ ]:
# problem = pybop.DesignProblem(
#     model, parameters, experiment, initial_state={"Initial SoC": 0.7}
# )
# cost = pybop.GravimetricEnergyDensity(problem)


builder = (
    pybop.builders.Pybamm()
    .set_simulation(
        model,
        parameter_values=parameter_set,
        experiment=experiment,
        initial_state=0.7,
    )
    .add_cost(pybop.costs.pybamm.GravimetricEnergyDensity())
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

Let's construct PyBOP's optimisation class. This class provides the methods needed to fit the forward model. For this example, we use particle swarm optimisation (PSO). Due to the computational requirements of the design optimisation methods, we limit the number of iterations to 15 for this example.

In [ ]:
# Set optimiser and options
options = pybop.PintsOptions(
    verbose=True,
    max_iterations=15,
)
optim = pybop.PSO(problem, options=options)

Finally, we run the optimisation and return the values obtained,

In [ ]:
results = optim.run()
print(f"Initial gravimetric energy density: {-results.initial_cost:.2f} Wh.kg-1")
print(f"Optimised gravimetric energy density: {-results.best_cost:.2f} Wh.kg-1")

Iter: 1 | Evals: 1 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 2 | Evals: 2 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 3 | Evals: 3 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 4 | Evals: 4 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 5 | Evals: 5 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 6 | Evals: 6 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 7 | Evals: 10 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 8 | Evals: 12 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 9 | Evals: 15 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |
Iter: 10 | Evals: 17 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04

## Plotting and Visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation.

### Comparing System Response

We can quickly plot the system's response using the estimated parameters compared to the initial parameters:


In [ ]:
# Update parameter values with the optimized results, SOC, and current function
identified_parameter_values = results.parameter_values

# Create simulation with the identified parameters
sim1 = pybamm.Simulation(
    model, parameter_values=identified_parameter_values, experiment=experiment
)
sol1 = sim1.solve()

# Create a new simulation with the original parameters
sim2 = pybamm.Simulation(model, parameter_values=parameter_set, experiment=experiment)
sol2 = sim2.solve()

# Plot the timeseries output
import plotly.graph_objects as go

fig = go.Figure(layout=go.Layout(title="Time-domain comparison", width=800, height=600))

fig.add_trace(
    go.Scatter(
        x=sol1.t, y=sol1["Voltage [V]"].data, mode="lines", name="Identified parameters"
    )
)

fig.add_trace(
    go.Scatter(
        x=sol2.t, y=sol2["Voltage [V]"].data, mode="lines", name="Original parameters"
    )
)

fig.update_layout(
    xaxis_title="Time / s",
    plot_bgcolor="white",
    paper_bgcolor="white",
)
fig.show()

### Cost Landscape

Finally, we can visualise the cost landscape and the path taken by the optimiser:


In [ ]:
pybop.plot.surface(optim);